# Generation of random catalogs from simulation

The idea is to create several catalogs from a whole set of simulated stars in one CCD (Axel's simulations).

Each catalog should contain:
- 50 test stars (without noise, non-shifted and one at each CCD-pixel)
- 50 stars to fit the model (with noise, shifted and randomly chosen)

### Imports

In [13]:
import numpy as np
import scipy.stats as sst
import matplotlib.pyplot as plt
import progressbar as pb
import os

from astropy.io import fits
from shapepipe.pipeline import file_io

### User parameters

In [14]:
# Location of the file containing raw simulated stars
CCD_number = 2
simu_path = '/Users/mschmitz/jbonnin/Data/CFIS_PSF'

# Directory used to store generated catalogs
output_path = '/Users/mschmitz/jbonnin/Data/CFIS_PSF/CCD-2_hqgklrsf'

# Path of a model file for creating .fits catalogs
basic_fits = '/Users/mschmitz/jbonnin/Data/all_w3_star_cat/star_selection-2079613-0.fits'

### Pre-defined functions

In [15]:
def create_fits(fits_path, data_dict, fits_ex = basic_fits):
    """
    Create a .fits file
    INPUTS:
        fits_path: path to the future .fits file
        data_dict: dictionnary where data is stored (usually {'VIGNET': ..., 'XWIN_IMAGE': ..., ...})
        fits_ex: path to an existing .fits file used as a model
    """
    fits_file = file_io.FITSCatalog(fits_path, open_mode = file_io.BaseCatalog.OpenMode.ReadWrite, SEx_catalog=True)
    fits_file.save_as_fits(data_dict, sex_cat_path = basic_fits)

def add_noise_std(images, sigma, seed = 0):
    N = images.shape[0]
    x_size = images.shape[1]
    y_size = images.shape[2]

    np.random.seed(seed)
    noise = np.array([sst.norm.rvs(scale = sigma, size = x_size * y_size) for k in range(N)])
    noisy_images = images + noise.reshape(N, x_size, y_size)

    return noisy_images

### Load the data

In [16]:
simu = np.load('{}/psf_ccd_{}.npy'.format(simu_path, CCD_number), allow_pickle = True).item()
print(simu.keys())

N_stars = len(simu['CCD_n'])
print('{} stars in CCD {}'.format(N_stars, CCD_number))

dict_keys(['vignet', 'e1_true', 'e2_true', 'fwhm', 'CCD_n', 'X', 'Y'])
24926 stars in CCD 2


In [17]:
# Identifying CCD-pixels

X_px = np.sort(list(set(simu['X'])))
Y_px = np.sort(list(set(simu['Y'])))

x_gap = int(X_px[1] - X_px[0])
print('Width of a CCD_pixel: {}'.format(x_gap))

y_gap = int(Y_px[1] - Y_px[0])
print('Height of a CCD_pixel: {}'.format(y_gap))

Width of a CCD_pixel: 409
Height of a CCD_pixel: 461


In [18]:
# Recovering index of stars present in each CCD-pixel

px_pos = [[[] for y in Y_px] for x in X_px]

for k in range(N_stars):
    for i in range(len(X_px)):
        for j in range(len(Y_px)):
            if (simu['X'][k] == X_px[i]) and (simu['Y'][k] == Y_px[j]):
                px_pos[i][j].append(k)

N_stars_pCCD = np.array([[len(stars) for stars in px_col] for px_col in px_pos])
print(np.array(N_stars_pCCD).T)

min_px = np.min(N_stars_pCCD)
print('Minimum number of stars per CCD-pixel: {}'.format(min_px))

[[495 507 506 478 492]
 [492 484 496 522 475]
 [493 506 509 474 453]
 [513 507 505 484 479]
 [493 508 501 485 505]
 [515 540 484 495 450]
 [510 525 470 503 515]
 [535 519 504 489 543]
 [509 510 516 478 450]
 [518 481 500 539 466]]
Minimum number of stars per CCD-pixel: 450


### Select stars

In [19]:
N_catalogs = min_px // 2

flat_idx = [idx for col in px_pos for idx in col]
slices_idx = [list(s) for s in zip(*flat_idx)]

test_slices_idx = slices_idx[:N_catalogs]
train_slices_idx = slices_idx[N_catalogs:-1]

train_idx = np.array(train_slices_idx).flatten()

### Build test catalogs

In [20]:
test_dicts = [{'VIGNET': simu['vignet'][s], 'XWIN_IMAGE': simu['X'][s], 'YWIN_IMAGE': simu['Y'][s]} for s in slices_idx[:N_catalogs]]

os.system('mkdir ' + output_path)
print('Building test stars catalogs ...')
# for n in pb.progressbar(range(N_catalogs)):
for n in range(N_catalogs):
    create_fits('{}/test-star_selection-{:07d}-{}.fits'.format(output_path, n, CCD_number), test_dicts[n])
print('Done')

Building test stars catalogs ...
Done


### Shift train stars

In [21]:
np.random.seed(0)
x_shifts = np.random.uniform(- x_gap / 2, x_gap / 2, len(train_idx))
y_shifts = np.random.uniform(- y_gap / 2, y_gap / 2, len(train_idx))

simu_s = dict(simu)

simu_s['X'][train_idx] = simu['X'][train_idx] + x_shifts
simu_s['Y'][train_idx] = simu['Y'][train_idx] + y_shifts

### Add noise

In [22]:
sig_noise = 1e-3

simu_sn = dict(simu_s)
simu_sn['vignet'][train_idx] = add_noise_std(simu_sn['vignet'][train_idx], sig_noise)

### Select train stars

In [23]:
np.random.seed(0)
rand_train_idx = np.copy(train_idx)
np.random.shuffle(rand_train_idx)
train_selections = [rand_train_idx[(50 * n):(50 * (n + 1))] for n in range(N_catalogs)]

### Build train catalogs

In [24]:
train_dicts = [{'VIGNET': simu_sn['vignet'][s], 'XWIN_IMAGE': simu_sn['X'][s], 'YWIN_IMAGE': simu_sn['Y'][s]} for s in train_selections]

os.system('mkdir ' + output_path)
print('Building train stars catalogs ...')
# for n in pb.progressbar(range(N_catalogs)):
for n in range(N_catalogs):
    create_fits('{}/train-star_selection-{:07d}-{}.fits'.format(output_path, n, CCD_number), train_dicts[n])
print('Done')

Building train stars catalogs ...
Done
